In [2]:
# %load lyrics_crawl_preprocessing.py
import argparse
import pandas as pd
import time
import re
from urllib.request import urlopen
from bs4 import BeautifulSoup
from pandas import DataFrame
from selenium import webdriver
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoAlertPresentException




parser = argparse.ArgumentParser()
parser.add_argument('begin_year', type=int, help='beginning year you wanna crawl')
parser.add_argument('end_year', type=int, help='end year you wanna crawl')
parser.add_argument('begin_month', type=int, help='beginning month you wanna crawl')
parser.add_argument('end_month', type=int, help='end month you wanna crawl')

args = parser.parse_args()

"""
begin_year = args.begin_year
end_year = args.end_year
begin_month = args.begin_month
end_month = args.end_month
"""

begin_year = 2019
end_year = 2019
begin_month = 1
end_month = 3

# track information sheet
years_list = []
months_list = []
weeks_list = []
ranks_list = []
artists_list = []
titles_list = [] # song's title
trackId_list = []
genre_list = []
release_list = [] # release_date
notes_list = [] # list for special case. classify track that 19+(for only adult), track that doesn't exist anymore, track that doesn't have lyric, track that doesn't provide service(can't play).
number_of_lines_list = [] # lyric's number of lines
entire_lyric_trackId_list = [] # list for entire lyric's track identifier
# 가사 분석을 순수 한글로만 이루어진 가사 데이터로만 진행하고 싶을때 사용하기 위함입니다.
has_alphabet_list = [] # check if lyric has alphabet

# lyric data sheet
lyrics_list = []

# lyric's sentences sheet
line_list = [] # index for sentence's line
sentence_data_list = [] # list for sentence data
lyrics_sentence_trackId_list = [] # list for sentence's track identifier 

# edit with your path of chromedriver
browser = webdriver.Chrome('/Users/pshkh/chromedriver_win32/chromedriver')

# enter the naver's login page
browser.get('https://nid.naver.com/nidlogin.login')

# if you wanna crawl 19+ song, you have to login
# 수동으로 네이버 로그인을 해줘야한다. 그것을 위한 10초 (캡차 회피 현존 방식은 전부 막힘)
time.sleep(10)

# period that you wanna crawl

"""
begin_year = args.begin_year
end_year = args.end_year
begin_month = args.begin_month
end_month = args.end_month
"""

begin_year = 2019
end_year = 2019
begin_month = 1
end_month = 3


# pattern that I wanna replace
rep = {'<br>':'\n', '<br/>':'\n', 'amp;':'', '-':' ', '_':'', '—':''}
# use these three lines to do the replacement
rep = dict((re.escape(k), v) for k, v in rep.items())
pattern = re.compile("|".join(rep.keys()))
def remove_tags(lyrics):
    # replace pattern that I defined above
    lyrics = pattern.sub(lambda m: rep[re.escape(m.group(0))], lyrics)
    # remove [text] and <text>    
    lyrics = re.sub("[\<\[].*?[\>\]]", "", lyrics)
    return lyrics

for year in range(begin_year, end_year+1, 1):
    for month in range(begin_month, end_month+1, 1):
        browser.get("https://music.naver.com/listen/history/index.nhn?type=TOTAL&year=%d&month=%02d&week=%d" % (year, month, 1))
        html = browser.page_source
        
        # Check 4 weeks per a month or 5 weeks per a month
        max_week = len(BeautifulSoup(html, "lxml").find_all('a', class_='_a_week'))-1

        for week in range(1, max_week+1, 1):
            # page 1 : Top 1~50, page 2 : Top 51~100
            for page in range(0, 2):
                browser.get("https://music.naver.com/listen/history/index.nhn?type=TOTAL&year=%d&month=%02d&week=%d&page=%d" % (year, month, week, page+1))
                html = browser.page_source

                rankSoup = BeautifulSoup(html, "lxml")
                
                # information about tracks
                tracks = rankSoup.find_all('tr', class_='_tracklist_move')
                
                # 서비스 중인 곡은 _title, 서비스 종료된 곡은 title
                songs = rankSoup.find_all(['a', 'span'], class_=re.compile(r"^(title|_title)$")) 
                artists = rankSoup.find_all('td', class_='_artist')

                rank = 1 # rank starts at 1
                for song in songs:
                    
                    # index : 0 Trackid, 2 서비스 유무, 4 19금 유무, 9 가사 유무
                    trackinfo = tracks[rank]['trackdata'].split('|') 

                    # for genre & release date
                    urldata = urlopen("http://music.naver.com/track/index.nhn?trackId=%s" % (trackinfo[0]))
                    trackSoup = BeautifulSoup(urldata, "html.parser")
                    try:
                        trackdata = trackSoup.find("dl", class_="desc").find_all("dd") # for genre & release date
                    except AttributeError:
                        # 트랙 없음
                        genre_list.append('')   
                        release_list.append('')
                    else: 
                        genre_list.append(trackdata[1].text[1:])
                        release_list.append(trackdata[2].text)

                    if trackinfo[9] == 'true':
                        browser.get("https://music.naver.com/lyric/index.nhn?trackId=%s" % (trackinfo[0]))
                        try:
                            Alert(browser).accept()
                            lyrics = ''  
                            lyrics_sentence_list = ['']    
                            note = '트랙 없음'

                            if trackinfo[4] == 'true':
                                note += '&19금'
                            if trackinfo[2] == 'false':
                                note += '&서비스 종료'
                        except NoAlertPresentException:
                            html = browser.page_source
                            lyricsSoup = BeautifulSoup(html, "html.parser")

                            lyrics = str(lyricsSoup.find('div', class_='show_lyrics'))
                            
                            lyrics = remove_tags(lyrics)
                            
                            # split with \n
                            lyrics_sentence_list = lyrics.split('\n')

                            # remove punctuation
                            lyrics_sentence_list = [re.sub(r'[^\w\s]', '', lyrics, re.UNICODE) for lyrics in lyrics_sentence_list]

                            '''
                            # remove row in lyric has alphabet. use this code if you wanna pure korean lyrics
                            # 영어로 된 문장 혹은 알파벳이 포함된 문장 전체를 제거합니다. 순수한글로만 이루어진 가사만 얻고 싶을때 사용합시다.
                            lyrics_sentence_list = list(filter(lambda w: not re.match(r'[a-zA-Z]+', w), lyrics_sentence_list)) 
                            '''

                            '''
                            # remove only english from row of lyric consist of kor and eng.
                            # 한영 혼용 문장에서 오직 영어만을 제거합니다. (한글 부분은 남습니다.) 
                            lyrics_sentence_list = [re.sub(r'[a-zA-Z]', '', lyrics) for lyrics in lyrics_sentence_list]
                            '''

                            # remove blank 
                            lyrics_sentence_list = list(filter(lambda a : a != '', lyrics_sentence_list))   
                            lyrics_sentence_list = list(filter(lambda a : a.isspace() != True, lyrics_sentence_list)) 
                            lyrics_sentence_list = list(filter(lambda a : a.lstrip(), lyrics_sentence_list))
                            lyrics_sentence_list = list(filter(lambda a : a.rstrip(), lyrics_sentence_list))

                            # remove blank from entire lyric
                            lyrics = '\n'.join(lyrics_sentence_list)

                            note = ''

                            if trackinfo[4] == 'true':
                                note = '19금'
                                if trackinfo[2] == 'false':
                                    note += '&서비스 종료'
                            else:
                                if trackinfo[2] == 'false':
                                    note = '서비스 종료'
                    else :
                        lyrics = ''   
                        lyrics_sentence_list = ['']    
                        note = '가사 등록 안됨'

                    artist = artists[rank]

                    # TrackInfo Sheet
                    years_list.append(year)
                    months_list.append(month)
                    weeks_list.append(week)
                    ranks_list.append(rank + 50*page)
                    artists_list.append(artist.text.strip())
                    titles_list.append(song.text)
                    trackId_list.append(trackinfo[0])
                    notes_list.append(note)
                    number_of_lines_list.append(len(lyrics_sentence_list))

                    # entire lyric
                    if trackinfo[0] not in entire_lyric_trackId_list:
                        entire_lyric_trackId_list.append(trackinfo[0])
                        lyrics_list.append(lyrics)

                    # lyric's sentence
                    if trackinfo[0] not in lyrics_sentence_trackId_list:
                        line = 1                           
                        for ly in lyrics_sentence_list:
                            lyrics_sentence_trackId_list.append(trackinfo[0])
                            sentence_data_list.append(ly)
                            line_list.append(line)
                            line +=1

                    check_has_alphabet = "False"
                    for ly in lyrics_sentence_list:
                        alpahbet_val = re.search('[a-zA-Z]+',ly)
                        if(alpahbet_val == None): 
                            pass
                        else:
                            if(alpahbet_val[0].isalpha()):
                                check_has_alphabet = "True"
                                break

                    has_alphabet_list.append(check_has_alphabet)

                    rank += 1 # increase rank, rank is in range 1~100

trackInfoSheet_column_list = {
'Year': years_list,
'Month': months_list,
'Week': weeks_list,
'Ranking': ranks_list,
'Artist': artists_list,
'Title': titles_list,
'Track_Id' : trackId_list,   
'Genre' : genre_list,
'Release' : release_list,
'Number_Of_Lines' : number_of_lines_list,
'Note' : notes_list,
'Has_English' : has_alphabet_list,
'Lyric': lyrics_list
}

lyricsDataSheet_column_list = {
'Track_Id' : entire_lyric_trackId_list,    
'Lyric': lyrics_list,   
}

SentenceSheet_column_list = {
'Track_Id' : lyrics_sentence_trackId_list,
'Lyric_Sentence' : sentence_data_list,
'Line' : line_list,
}

df_track_info = pd.DataFrame.from_dict(trackInfoSheet_column_list, orient='index')
df_track_info = df_track_info.transpose()

df_lyric_data = pd.DataFrame.from_dict(lyricsDataSheet_column_list, orient='index')
df_lyric_data = df_lyric_data.transpose()

df_sentence_data = pd.DataFrame.from_dict(SentenceSheet_column_list, orient='index')
df_sentence_data = df_sentence_data.transpose()

writer = pd.ExcelWriter("2019년 1월.xlsx")

df_track_info.to_excel(writer, sheet_name='Top100_chart_track_info', startrow=1, header=True)
df_lyric_data.to_excel(writer, sheet_name='Top100_lyrics_data', startrow=1, header=True)
df_sentence_data.to_excel(writer, sheet_name='Sentence_data', startrow=1, header=True)

writer.save()

usage: ipykernel_launcher.py [-h] begin_year end_year begin_month end_month
ipykernel_launcher.py: error: argument begin_year: invalid int value: 'C:\\Users\\Ina Lee\\AppData\\Roaming\\jupyter\\runtime\\kernel-6b31b3f5-cec7-4bcf-900a-89d7bcebacb8.json'


SystemExit: 2